<a href="https://colab.research.google.com/github/revathi-9525/text-to-image/blob/main/text_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🚀 Stable Diffusion with Gradio in Colab
!pip install diffusers transformers accelerate safetensors gradio --quiet

import gradio as gr
from diffusers import StableDiffusionPipeline
import torch
import random
import os
import datetime

# ---------------- Model Setup ---------------- #
model_id = "dreamlike-art/dreamlike-diffusion-1.0"
device = "cuda" if torch.cuda.is_available() else "cpu"
# dtype = torch.float16 if device == "cuda" else torch.float32

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32, low_cpu_mem_usage=False)
pipe = pipe.to(device)

# ---------------- Enhancer + Styles ---------------- #
style_prompts = {
    "Cartoon": "in cartoon style, colorful, playful",
    "Ghibli": "in Studio Ghibli style, whimsical, detailed backgrounds",
    "Anime": "in anime style, vibrant, sharp lines",
    "Realistic": "ultra realistic, photorealistic details",
    "Cyberpunk": "cyberpunk, neon lights, futuristic city",
    "None": ""
}

aspect_ratios = {
    "1:1": (512, 512),
    "3:4": (512, 768),   # default
    "16:9": (768, 432)
}

# Store history as list of dicts
history = []


def generate_images(prompt, style, aspect, num_images, seed, cfg_scale, enhance):
    global history

    # Prompt Enhancer
    if enhance:
        prompt = f"{prompt}, highly detailed, 4k, trending on ArtStation, ultra sharp"

    # Style injection
    if style in style_prompts and style != "None":
        prompt = f"{prompt}, {style_prompts[style]}"

    width, height = aspect_ratios[aspect]

    # Seed
    if seed is None or seed == "":
        seed = random.randint(0, 999999)
    generator = torch.manual_seed(int(seed))

    # Generate
    new_images = []
    for i in range(num_images):
        image = pipe(
            prompt,
            height=height,
            width=width,
            guidance_scale=cfg_scale,
            generator=generator
        ).images[0]

        # Save
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"output_{timestamp}_{i}.png"
        image.save(filename)

        record = {
            "prompt": prompt,
            "style": style,
            "aspect": aspect,
            "seed": seed,
            "cfg_scale": cfg_scale,
            "filename": filename
        }

        new_images.append(image)  # for gallery
        history.append(record)

    # ✅ return both table + gallery history
    history_table = [
        [h["prompt"], h["style"], h["aspect"], str(h["seed"]), h["cfg_scale"], h["filename"]]
        for h in history
    ]
    history_gallery = [h["filename"] for h in history]

    return new_images, history_table, history_gallery


def delete_image(index):
    global history
    if 0 <= index < len(history):
        record = history.pop(index)
        if os.path.exists(record["filename"]):
            os.remove(record["filename"])

    history_table = [
        [h["prompt"], h["style"], h["aspect"], str(h["seed"]), h["cfg_scale"], h["filename"]]
        for h in history
    ]
    history_gallery = [h["filename"] for h in history]
    return history_table, history_gallery


def clear_history():
    global history
    for record in history:
        if os.path.exists(record["filename"]):
            os.remove(record["filename"])
    history = []
    return [], []


# ---------------- Gradio UI ---------------- #
with gr.Blocks() as demo:
    gr.Markdown("# 🎨 AI Image Generator (Stable Diffusion)")
    gr.Markdown("Customize your prompt, style, aspect ratio, and more. Generate, download, and manage your images!")

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="Enter your prompt", placeholder="e.g. A castle on a floating island")
            enhance = gr.Checkbox(label="🔮 Enhance Prompt", value=True)
            style = gr.Dropdown(list(style_prompts.keys()), label="Style (default=None)", value="None")
            aspect = gr.Radio(list(aspect_ratios.keys()), label="Aspect Ratio", value="3:4")
            num_images = gr.Slider(1, 5, value=1, step=1, label="Batch Generation (# of images)")
            seed = gr.Textbox(label="Seed (leave blank for random)", value="")
            cfg_scale = gr.Slider(1, 15, value=7.5, step=0.5, label="CFG Scale (Creativity vs Prompt-Adherence)")
            generate_btn = gr.Button("🚀 Generate")

        with gr.Column(scale=2):
            output_gallery = gr.Gallery(label="Generated Images", show_label=True, columns=2, height=400)

    with gr.Row():
        gr.Markdown("## 📜 History")

    with gr.Row():
        history_table = gr.Dataframe(
            headers=["Prompt", "Style", "Aspect", "Seed", "CFG Scale", "Filename"],
            datatype=["str", "str", "str", "str", "number", "str"],
            label="History Table"
        )

    with gr.Row():
        history_gallery = gr.Gallery(label="History Images", columns=4, height=300)

    with gr.Row():
        delete_index = gr.Number(label="Delete History Index", value=0)
        delete_btn = gr.Button("🗑 Delete Selected Image")
        clear_btn = gr.Button("🔥 Clear All History")

    # Actions
    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, style, aspect, num_images, seed, cfg_scale, enhance],
        outputs=[output_gallery, history_table, history_gallery]
    )

    delete_btn.click(fn=delete_image, inputs=delete_index, outputs=[history_table, history_gallery])
    clear_btn.click(fn=clear_history, outputs=[history_table, history_gallery])

# 👇 Works in Colab
demo.launch(share=True)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--dreamlike-art--dreamlike-diffusion-1.0/snapshots/9fb5a6463bf79d81152e715e8d2a8b988f96c790/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e20e0020b07e8f3ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
